In [2]:
import torch
from torch import nn
import os
import sys
sys.path.append('../')
import utils.csv as csv
from torchmetrics.classification import MultilabelF1Score
import pandas as pd

d:\LenovoSoftstore\Anaconda\envs\yolov5\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
# file path
PATH = os.path.join('..', 'outputs', 'csv','multi_label', '04')
model = 'transformer'
ref_path = os.path.join(PATH, f'{model}_8ml_ref.csv')
pred_path = os.path.join(PATH, f'{model}_8ml_pred.csv')
class_path = 'D:\\Deutschland\\FUB\\master_thesis\\data\\ref\\all\\8_main_classes.csv'

In [25]:
ref = csv.load(ref_path, 'id')
pred = csv.load(pred_path, 'id')
classes = csv.load(class_path, 'id')
res = pred == ref
res['sum'] = res.sum(axis=1)
res.sort_values(by='sum', ascending=False, inplace=True)
output = pd.merge(res, classes, on='id', how='inner')
out_path = os.path.join(PATH, f'{model}_8ml_count.csv')
csv.export(output, out_path, True)

export file ..\outputs\csv\multi_label\04\transformer_8ml_count.csv
